# Modeling and solving search problems

Consider the following river crossing puzzle:

## Jealous Husbands Problem with an Island

There are n couples: $(𝐻_1,𝑊_1)$, $(𝐻_2,𝑊_2)$, …, $(𝐻_𝑛,𝑊_𝑛)$  where $𝐻_𝑖$ is husband $𝑖$ and $𝑊_𝑖$ is wife $𝑖$.

They need to cross from the left bank of a river to the right bank.

There is a boat that can carry at most two people at a time.

Important: There is an island midway between the left and right banks. The boat can stop at the island.

Rules:

1. Jealousy constraint: No wife may ever be in the presence of another husband (on any shore or the island or in the boat) unless her own husband is also present. Otherwise, fights (and failures) occur.

2. The boat needs at least one person to operate (no remote-controlled trips).

3. The boat can travel between:

            Left bank ↔ Island

            Island ↔ Right bank

4. All people, including the boat, start at the left bank.

Goal: get everyone (husbands and wives) safely to the right bank, obeying all the rules.


## Tasks:

1. Put the problem in a form of a search tree: root, nodes, solution and rules for constructing the childrens.
2. Solve it usinh a classic DFS
3. Solve it using a classic Back-tracking.
4. Solve it using the A* method.  

In [20]:
# 1
LEFT, ISLAND, RIGHT = 0, 1, 2

class State:
    def __init__(self, husbands, wives, boat):
        self.husbands = husbands.copy()
        self.wives = wives.copy()
        self.boat = boat  # 0=left, 1=island, 2=right

    def __eq__(self, other):
        return (self.husbands == other.husbands and 
                self.wives == other.wives and 
                self.boat == other.boat)
    def __lt__(self, other):
        return self.boat < other.boat

    def __hash__(self):
        return hash((tuple(self.husbands), tuple(self.wives), self.boat))

    def __str__(self):
        location_names = ["Left", "Island", "Right"]
        result = f"Boat: {location_names[self.boat]}\n"
        for i in range(len(self.husbands)):
            result += f"Couple {i}: Husband at {location_names[self.husbands[i]]}, Wife at {location_names[self.wives[i]]}\n"
        return result

    def is_valid(self):
        for loc in [LEFT, ISLAND, RIGHT]:
            wives_here = [i for i in range(len(self.wives)) if self.wives[i] == loc]
            if not wives_here:
                continue
            husbands_here = [i for i in range(len(self.husbands)) if self.husbands[i] == loc]
            for w in wives_here:
                if self.husbands[w] != loc and husbands_here:
                    return False
        return True

    def is_goal(self, n):
        return all(self.husbands[i] == RIGHT and self.wives[i] == RIGHT for i in range(n))


def move_people(state, people, destination):
    new_husbands = state.husbands.copy()
    new_wives = state.wives.copy()

    for person_type, idx in people:
        if person_type == 'H':
            new_husbands[idx] = destination
        else:
            new_wives[idx] = destination

    return State(new_husbands, new_wives, destination)


def get_successors(state, n):
    successors = []

    people_at_boat = []
    for i in range(n):
        if state.husbands[i] == state.boat:
            people_at_boat.append(('H', i))
        if state.wives[i] == state.boat:
            people_at_boat.append(('W', i))
            
    # next location available
    destinations = {
        LEFT: [ISLAND],
        ISLAND: [LEFT, RIGHT],
        RIGHT: [ISLAND]
    }[state.boat]

    for dest in destinations:
        for person in people_at_boat:
            new_state = move_people(state, [person], dest)
            if new_state.is_valid():
                successors.append(new_state)

        for i in range(len(people_at_boat)):
            for j in range(i + 1, len(people_at_boat)):
                persons = [people_at_boat[i], people_at_boat[j]]
                new_state = move_people(state, persons, dest)
                if new_state.is_valid():
                    successors.append(new_state)

    return successors

In [2]:
# 2 DFS solution
def dfs_solution(n, track_visited=True):
    initial_state = State([LEFT] * n, [LEFT] * n, LEFT)
    visited = set()

    def dfs(state, path):
        if state.is_goal(n):
            return path + [state]

        visited.add(state)
        successors = get_successors(state, n)

        def priority_score(s):
            return sum(1 for h in s.husbands if h == RIGHT) + sum(1 for w in s.wives if w == RIGHT)

        successors.sort(key=priority_score, reverse=True)

        for succ in successors:
            if succ not in visited:
                result = dfs(succ, path + [state])
                if result:
                    return result

        if not track_visited:
            visited.remove(state)
        return None

    return dfs(initial_state, [])


In [3]:
def print_solution(path, algorithm_name):
    if not path:
        print(f"\n No solution found using {algorithm_name}!")
        return

    print(f"\n {algorithm_name} solution found in {len(path) - 1} moves.")

    for i, state in enumerate(path):
        if i == 0:
            print("\nInitial state:")
        else:
            print(f"\nMove {i}:")

            prev = path[i - 1]
            moved = []

            for j in range(len(prev.husbands)):
                if prev.husbands[j] != state.husbands[j]:
                    moved.append(f"Husband {j}")
                if prev.wives[j] != state.wives[j]:
                    moved.append(f"Wife {j}")

            locations = ["Left", "Island", "Right"]
            direction = f"{locations[prev.boat]} → {locations[state.boat]}" if prev.boat < state.boat else f"{locations[prev.boat]} ← {locations[state.boat]}"
            print(f"  Moved: {', '.join(moved)}")
            print(f"  Boat: {direction}")

        print(state)


In [5]:
def print_dfs_solution(n):
    print("\n DFS solution")
    sol = dfs_solution(n)
    print_solution(sol, "Depth-First Search")

In [6]:
print_dfs_solution(3)


 DFS solution

 Depth-First Search solution found in 56 moves.

Initial state:
Boat: Left
Couple 0: Husband at Left, Wife at Left
Couple 1: Husband at Left, Wife at Left
Couple 2: Husband at Left, Wife at Left


Move 1:
  Moved: Husband 0, Wife 0
  Boat: Left → Island
Boat: Island
Couple 0: Husband at Island, Wife at Island
Couple 1: Husband at Left, Wife at Left
Couple 2: Husband at Left, Wife at Left


Move 2:
  Moved: Husband 0, Wife 0
  Boat: Island → Right
Boat: Right
Couple 0: Husband at Right, Wife at Right
Couple 1: Husband at Left, Wife at Left
Couple 2: Husband at Left, Wife at Left


Move 3:
  Moved: Husband 0
  Boat: Right ← Island
Boat: Island
Couple 0: Husband at Island, Wife at Right
Couple 1: Husband at Left, Wife at Left
Couple 2: Husband at Left, Wife at Left


Move 4:
  Moved: Husband 0
  Boat: Island ← Left
Boat: Left
Couple 0: Husband at Left, Wife at Right
Couple 1: Husband at Left, Wife at Left
Couple 2: Husband at Left, Wife at Left


Move 5:
  Moved: Wife 1
  

In [11]:
# 3 Backtracking solution
def bkt_solution(n):
    initial_state = State([LEFT] * n, [LEFT] * n, LEFT)
    visited_on_this_path = set()

    def bkt_generator(current_state, path_history):
        if current_state.is_goal(n):
            return path_history + [current_state]

        visited_on_this_path.add(current_state)

        for successor in get_successors(current_state, n):
            if successor not in visited_on_this_path:
                result = bkt_generator(successor, path_history + [current_state])
                if result:
                    return result

        visited_on_this_path.remove(current_state)
        return None

    return bkt_generator(initial_state, [])

In [12]:
def print_bkt_solution(n):
    print("\n Backtracking solution")
    sol = bkt_solution(n)
    print_solution(sol, "Backtracking")

In [13]:
print_bkt_solution(3)


 Backtracking solution

 Backtracking solution found in 206 moves.

Initial state:
Boat: Left
Couple 0: Husband at Left, Wife at Left
Couple 1: Husband at Left, Wife at Left
Couple 2: Husband at Left, Wife at Left


Move 1:
  Moved: Husband 0, Wife 0
  Boat: Left → Island
Boat: Island
Couple 0: Husband at Island, Wife at Island
Couple 1: Husband at Left, Wife at Left
Couple 2: Husband at Left, Wife at Left


Move 2:
  Moved: Husband 0
  Boat: Island ← Left
Boat: Left
Couple 0: Husband at Left, Wife at Island
Couple 1: Husband at Left, Wife at Left
Couple 2: Husband at Left, Wife at Left


Move 3:
  Moved: Wife 1
  Boat: Left → Island
Boat: Island
Couple 0: Husband at Left, Wife at Island
Couple 1: Husband at Left, Wife at Island
Couple 2: Husband at Left, Wife at Left


Move 4:
  Moved: Wife 0
  Boat: Island ← Left
Boat: Left
Couple 0: Husband at Left, Wife at Left
Couple 1: Husband at Left, Wife at Island
Couple 2: Husband at Left, Wife at Left


Move 5:
  Moved: Husband 1
  Boat: Le

In [22]:
# 4 A* solution
import heapq

def a_star_solution(n):
    initial_state = State([LEFT] * n, [LEFT] * n, LEFT)

    def h_values(state):
        return sum(1 for h in state.husbands if h != RIGHT) + sum(1 for w in state.wives if w != RIGHT)
        
    queue = [(h_values(initial_state), 0, initial_state, [])]
    visited = {initial_state: 0}

    while queue:
        _, g, state, path = heapq.heappop(queue)

        if state.is_goal(n):
            return path + [state]

        for successor in get_successors(state, n):
            new_g = g + 1

            if successor not in visited or new_g < visited[successor]:
                visited[successor] = new_g
                f = new_g + h_values(successor)
                heapq.heappush(queue, (f, new_g, successor, path + [state]))

    return None


In [23]:
def print_a_star_solution(n):
    print("\n A* solution")
    sol = a_star_solution(n)
    print_solution(sol, "A*")

In [24]:
print_a_star_solution(3)


 A* solution

 A* solution found in 18 moves.

Initial state:
Boat: Left
Couple 0: Husband at Left, Wife at Left
Couple 1: Husband at Left, Wife at Left
Couple 2: Husband at Left, Wife at Left


Move 1:
  Moved: Husband 2, Wife 2
  Boat: Left → Island
Boat: Island
Couple 0: Husband at Left, Wife at Left
Couple 1: Husband at Left, Wife at Left
Couple 2: Husband at Island, Wife at Island


Move 2:
  Moved: Husband 2
  Boat: Island ← Left
Boat: Left
Couple 0: Husband at Left, Wife at Left
Couple 1: Husband at Left, Wife at Left
Couple 2: Husband at Left, Wife at Island


Move 3:
  Moved: Wife 0, Wife 1
  Boat: Left → Island
Boat: Island
Couple 0: Husband at Left, Wife at Island
Couple 1: Husband at Left, Wife at Island
Couple 2: Husband at Left, Wife at Island


Move 4:
  Moved: Wife 0
  Boat: Island ← Left
Boat: Left
Couple 0: Husband at Left, Wife at Left
Couple 1: Husband at Left, Wife at Island
Couple 2: Husband at Left, Wife at Island


Move 5:
  Moved: Husband 1, Husband 2
  Boat: 